# Ever want to create a Google Form with certain fields pre-filled from a CSV?

This notebook will go through how to:

1.   Generate a google sheet with pre-filled forms links from a CSV

The fake data used for this notebook was generated via [Mockaroo](https://www.mockaroo.com/)

## Set-up
1.   Generate a pre-filled form [link](https://support.google.com/a/users/answer/9308781?hl=en#)
2.   Get the names of the columns from your CSV that you would like to use for the forms pre-filled values
3.   Edit the base form link, column values, and replace method values

Once you are set-up and ready, navigate to the Menu bar, select Runtime, then click Run All

## Import libraries

In [10]:
# Import libraries
from google.colab import auth
from oauth2client.client import GoogleCredentials as GC
from gspread_dataframe import set_with_dataframe
from datetime import datetime
from google.colab import files
import gspread
import pandas as pd
import io
import itertools
from tqdm import tqdm

## Authenticate your gmail account

In [14]:
# Authenticate gmail
auth.authenticate_user()
gc = gspread.authorize(GC.get_application_default())

## Upload your CSV file

In [11]:
# Import CSV file
uploaded = files.upload()

# Grab file name from upload
file_name = list(uploaded.keys())[0] 

# Create dataframe from CSV
df = pd.read_csv(io.BytesIO(uploaded[file_name]))

Saving MOCK_DATA.csv to MOCK_DATA.csv


## Displaying the columns names for reference

In [12]:
# iterating the columns names for reference
for col in df.columns: 
    print(col) 

id
first_name
last_name
address
city
state
zip
age
email


## Change the base form link, column values, and replace method values

In [15]:
# Add pre-filled form links to list

base_form_link = 'https://docs.google.com/forms/d/e/1FAIpQLScFX9rGNpZuO83zwRcHY5onn68vfHKXklUyHhaYAMrO_0wySQ/viewform?usp=pp_url&entry.1787852589=00000&entry.1998997249=John+Doe%0A123+Apartment+Way%0A67'
results = []

# Create a list of pre-filled form links by iterating through dataframe and performing a replace on the base form link for each row
for index, row in df.iterrows():
  
  # COLUMN VALUES TO CHANGE
  id = str(row['id']) #<--------------
  first = "Name: " + row['first_name'] #<--------------
  last = row['last_name'] #<--------------
  address = "Address: " + row['address'] #<--------------
  age = "Age: " + str(row['age']) #<--------------

  # REPLACE VALUES TO CHANGE
  form_link = base_form_link.replace("00000", id).replace("John", first).replace("Doe", last).replace("123+Apartment+Way", address).replace("67", age).replace(" ","+") #<--------------
  
  results.append(form_link)

# Add form link list into dataframe
df['Form Links'] = pd.Series( (v for v in results) )

# Get todays date
dt = datetime.date(datetime.now())

# Create google sheets from sub dataframe
with tqdm(total=1) as pbar:
  title = '{}_{}'.format(dt,file_name)
  gc.create(title)  # if not exist
  sheet = gc.open(title).sheet1
  set_with_dataframe(sheet, df.reset_index(drop=True))
  pbar.update(1)
print('\n\nYour Google Sheets have been created!')

100%|██████████| 1/1 [00:05<00:00,  5.61s/it]



Your Google Sheets have been created!
